# Day 82: Medical Record Agent (Safe)

Medical records contain highly sensitive Protected Health Information (PHI). Under regulations like HIPAA (USA) or GDPR (EU), this data must be strictly protected. When using AI for medical analysis, we must ensure that no PHI is leaked to the model provider.

In this lab, we implement a **Medical Record Privacy Manager** to:
1. **Role-Based Access Control (RBAC)**: Ensuring only authorized personnel can trigger analysis.
2. **De-identification (Redaction)**: Using regex to strip Names, Birthdates, SSNs, and Phone Numbers from text.
3. **Tokenized Anonymization**: Replacing PHI with placeholders (e.g., `[NAME_0]`) that can be mapped back locally if needed.

In [ ]:
import sys
import os

# Add root directory to sys.path
sys.path.append(os.path.abspath('../../'))

from src.privacy.medical_records import MedicalRecordPrivacyManager, MedicalReport

## 1. Setup Sample Data

We define a raw medical report with several types of PHI.

In [ ]:
raw_text = """
Patient: John Doe
DOB: 05/12/1975
SSN: 111-22-3333
Contact: (555) 123-4567

Observed symptoms: Persistent cough and fatigue for 2 weeks.
Recommendation: Schedule a follow-up chest X-ray.
"""

report = MedicalReport(raw_text, patient_id="P9988", authorized_users=["DrHouse", "NurseRatched"])

## 2. Authorization and Sanitation

We simulate an authorized request to prepare the data for an external AI model.

In [ ]:
manager = MedicalRecordPrivacyManager()

print("--- AI Sanitation Pipeline ---")
clean_text = manager.sanitize_for_ai(report, user_id="DrHouse")
print(clean_text)

## 3. Handling Unauthorized Access

Observe how the system blocks an unauthorized user before de-identification even begins.

In [ ]:
blocked_attempt = manager.sanitize_for_ai(report, user_id="RandoInternetUser")
print(f"Access Result: {blocked_attempt}")

## 4. Reverse Mapping (Local Only)

The manager maintains a local mapping used to restore the original data after AI processing (if the AI returns comments keyed to the placeholders).

In [ ]:
_, mapping = manager.de_identify(raw_text)
import json
print("Secure Local Mapping (Never sent to AI):")
print(json.dumps(mapping, indent=2))